# Solution of the 1D Poisson problem using our own FEM code

In [ ]:
import import_hack

In [ ]:
from methodsnm.mesh import *
from methodsnm.visualize import *

In [ ]:
ne = 10

In [ ]:
  
mesh = Mesh1D((0,1),ne)
DrawMesh1D(mesh)

On the mesh, we define a finite element space:

In [ ]:
from methodsnm.fes import *
p1fes = P1_Segments_Space(mesh)
DrawShapes(p1fes)

On the mesh, with the given finite element space, we define the variational formulation:
$$
\int_{\Omega} \nabla u \cdot \nabla v dx + \int_{\Omega} u v dx = \int_{\Omega} f v dx
$$
Replacing 
* $u \leadsto u_h = \sum_j u_j \phi_j $ and
* $v \leadsto v_h = \phi_i$
we obtain a linear system:

In [ ]:
from methodsnm.forms import *
from methodsnm.formint import *

try: #include solution module (if exists)
    from methodsnm.solution import *
except:
    pass

blf = BilinearForm(p1fes)
c = GlobalFunction(lambda x: 1, mesh = mesh)
blf += LaplaceIntegral(c)
blf += MassIntegral(c)
blf.assemble()

lf = LinearForm(p1fes)
f = GlobalFunction(lambda x: np.sin(x), mesh = mesh)
lf += SourceIntegral(f)
lf.assemble()

blf.matrix

In [ ]:
print(blf.matrix, "\n", lf.vector)

Next, we solve the linear system. And we use a direct solver from scipy for that (for now):

In [ ]:
uh = FEFunction(p1fes)
from scipy.sparse.linalg import spsolve
uh.vector = spsolve(blf.matrix, lf.vector)

Finally, we can visualize the solution:

In [ ]:
DrawFunction1D(uh)

This problem now contained several simplifications, that can be removed step by step:
 * qualitative evaluation only (no quantitative evaluation, no convergence study)
 * boundary conditions are natural. Different boundary conditions make it more difficult
 * 1D $\leadsto$ 2D
 * P1 (low order) discretization vs. P2 and higher order discretizations